In [ ]:
# xmlrpc_server.ipynb

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
from xmlrpc import client as xc
import datetime
import pandas as pd
import pickle


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

server.register_introspection_functions()

stats_server = xc.ServerProxy("http://localhost:8018")

# Добавление в лог через сервер
def add_log(log_line):
    try:
        stats_server.add_log(log_line)
    except:
        return False
    return True

# Тест
def ping():
    add_log('ping')
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    add_log('now')
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    add_log('type')
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    add_log('sum')
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    add_log('pow')
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    add_log('black_list_check')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check_fio(sname, name, patronim, date):
    add_log('black_list_check_fio')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    result = frame.query('Surname == @sname and Name == @name and Patronym == @patronim and Birth == @date')
    if (result.empty):
        return f'{sname} {name} {patronim}, {date} : good boy'
    else:
        return f'{sname} {name} {patronim}, {date} : bad boy'

server.register_function(black_list_check_fio, 'black_list_check_fio')

# Бинарная передача данных
def send_back_binary(bin_data):
    add_log('send_back_binary')
    data = bin_data.data
    return xc.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

#Бинаризация
def binarization(bin_data, threshold):
    add_log('binarization')
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    
    if len(img_arr.shape) == 3:
        for i in range(height):
            for j in range(width):
                for k in range(3):
                    img_arr[i][j][k] = (img_arr[i][j][k] > threshold[k]) * 255
    elif len(img_arr.shape) == 2:
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = (img_arr[i][j] > threshold) * 255
    
    pimg = pickle.dumps(img_arr)        
    return xc.Binary(pimg)
    
server.register_function(binarization, 'binarization')
    
# Отражение по горизонтали 
def reflection_horizontal(bin_data):
    add_log('reflection_horizontal')
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    
    if len(img_arr.shape) == 3:
        for i in range(height):
            for j in range(width//2):
                for k in range(3):
                    img_arr[i][j][k], img_arr[i][width-1-j][k] = img_arr[i][width-1-j][k], img_arr[i][j][k]
    elif len(img_arr.shape) == 2:
        for i in range(height):
            for j in range(width//2):
                img_arr[i][j], img_arr[i][width-1-j]  = img_arr[i][width-1-j], img_arr[i][j]
    
    pimg = pickle.dumps(img_arr)        
    return xc.Binary(pimg)
    
server.register_function(reflection_horizontal, 'reflection_horizontal')

# Отражение по вертикали
def reflection_vertical(bin_data):
    add_log('reflection_vertical')
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    
    if len(img_arr.shape) == 3:
        for i in range(height//2):
            for j in range(width):
                for k in range(3):
                    img_arr[i][j][k], img_arr[height-i-1][j][k] = img_arr[height-i-1][j][k], img_arr[i][j][k]
    elif len(img_arr.shape) == 2:
        for i in range(height//2):
            for j in range(width):
                img_arr[i][j], img_arr[height-i-1][j] = img_arr[height-i-1][j], img_arr[i][j]
    
    pimg = pickle.dumps(img_arr)        
    return xc.Binary(pimg)
    
server.register_function(reflection_vertical, 'reflection_vertical')

# Инверсия цвета
def send_back_inversion(bin_data):
    add_log('send_back_inversion')
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]   
    
    if len(img_arr.shape) == 3:
        for i in range(height):
            for j in range(width):
                for k in range(3):
                    img_arr[i][j][k] = 255 - img_arr[i][j][k]
    elif len(img_arr.shape) == 2:
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = 255 - img_arr[i][j]
            
    
    pimg = pickle.dumps(img_arr)        
    return xc.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')


print("Listening on port 8008...")
server.serve_forever()


Listening on port 8008...


127.0.0.1 - - [07/Nov/2023 00:09:04] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:08] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:12] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:16] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:21] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:25] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:29] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:33] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:37] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:41] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:45] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:49] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:09:57] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:10:08] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 00:10:23] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/202